##### Importando bibliotecas

In [ ]:
import matplotlib.pyplot as plt # Criar gráficos e visualizações de dados
import glob # Encontra arquivos que correspondam a um padrão
import shutil # Operações em arquivos/diretórios
import csv # Importa a biblioteca csv para leitura/escrita de arquivos CSV
import pandas as pd # Oferece ferramentas para manipulação e análise de dados
import os # Oferece uma maneira de realizar operações relacionadas a arquivos do sistema operacional
from statistics import mean # Calcular a média de um conjunto de valores numéricos
import numpy as np # Funções para realizar operações matemáticas e numéricas de alto desempenho
from scipy.signal import welch #Usada para estimar a densidade espectral de potência (PSD) de um sinal
from moviepy.editor import ImageSequenceClip, VideoFileClip, concatenate_videoclips #Usadas para criar e manipular vídeos

##### Transformando arquivos em CSV e movendo para pasta específica

In [ ]:
"""
Ar -> Phase 1.Volume Fraction == 1
Agua -> Phase 1.Volume Fraction == 0
##########################################
Code:
0 -> cell index
1 -> x 
2 -> y
3 -> z
4 -> u
5 -> v
6 -> w
7 -> phase-1-vof gas
"""

plt.rcParams['text.usetex'] = True # Utilizar fórmulas em LaTeX na visualização de gráficos

data_time_solver = glob.glob('data/Data_level_ts-*') # Lista de arquivos que correspondem ao padrão 'data/Data_level_ts-*'

# Função para organizar os arquivos de acordo com o padrão de nomenclatura e transformá-los em csv, movendo-os para uma nova localização
def file_organizer(data_time_solver):
    for i in range(len(data_time_solver)): # Loop que percorre todos os caminhos de arquivos encontrados anteriormente na lista data_time_solver
        if len(data_time_solver[i][19:]) == 4: # Verifica se os últimos quatro caracteres do caminho do arquivo atual têm um comprimento igual a 4. A parte [19:] do caminho do arquivo é usada para ignorar os primeiros 19 caracteres e obter apenas os caracteres restantes
            location = f'data/data_csv/Data_level_ts_00{data_time_solver[i][19:]}.csv'
        else:
            location = f'data/data_csv/Data_level_ts_0{data_time_solver[i][19:]}.csv'

        shutil.move(data_time_solver[i], location) # Mover o arquivo atual para a nova localização definida anteriormente

file_organizer(data_time_solver) # Chama a função de organização dos arquivos

data_time = glob.glob('data/data_csv/Data_level_ts*.csv') # Lista de arquivos que correspondem ao padrão 'data/data_csv/Data_level_ts∗.csv'

#### Posição fixa

##### Função para calcular o nível no tempo

In [ ]:
# Declaração de variáveis
D = 0.05248 # Diâmetro da tubulação
Time = 5 # Time inicial
flow_time = 0.01 # Passo de tempo

Time_func = [] # Lista vazia para armazenar os valores de tempo

# Esta função recebe um DataFrame 'Z_values' contendo os dados para a posição escolhida
def Level_t(Z_values, Time):
    alpha = [row[7] for row in Z_values[0:]]  # Extrai a coluna 7 de Z_values e armazena em 'alpha'
    # Itera sobre as linhas de Z_values com um índice 'i' começando em 1
    for i, row in enumerate(Z_values[0:], start=0): # Loop que percorre os elementos de Z_values, começando do segundo elemento. O loop é executado uma vez para cada elemento na lista. Ingora o primeiro elemento de Z_values porque é o cabeçalho?. Enumerate acessa o índice e o valor simultaneamente
        if round(row[7], 1) == 1.00 and row[7] != Z_values[i - 1][7]: # Se o valor arredondado de row[7] for 1.00 e for diferente do valor anterior
            break
        elif i == len(Z_values) - 1: # Se for a última iteração do loop
            break

    Time_func.append(tuple([float(row[2]), float(Time), float(np.mean(alpha))])) # Adiciona uma tupla à lista Time_func contendo os valores de posição radial, Tempo e média dos níveis

##### Loop para processar cada arquivo em data_time

In [ ]:
# POSIÇÃO Z
for file in data_time:
    data = pd.read_csv(file)  # Lê o arquivo CSV usando pandas
    
    data_m = data.to_numpy()  # Converte o DataFrame em uma matriz numpy
    Time = round(Time + flow_time, 2)  # Atualiza o valor da variável Time

    # Escolher a posição axial que irá analisar por meio da posição mais próxima na malha
    #Z_pos = 3.005882263 #57D
    #Z_pos = 3.994117737 #76D
    #Z_pos = 4.998823643 #95D
    #Z_pos = 6.003529549 #114D
    #Z_pos = 7.008234978 #133D
    #Z_pos = 7.996470451 #152D
    Z_pos = 9.001176834 #171D

    rounded_Z_pos = round(Z_pos, 4) # Arredonda a posição para 4 casas decimais
    Z_values = [row for row in data_m if round(row[3], 4) == rounded_Z_pos] # Filtrar os valores com base na coordenada Z usando NumPy
    
    Level_t(Z_values, Time)

##### Visualização dos dados

In [ ]:
# Função para cálculo da média móvel de entrada x usando uma janela de tamanho w
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w 
    # O resultado da convolução é uma estimativa da média móvel, e a divisão pelo tamanho da janela w normaliza os valores para obter a média

In [ ]:
def calculate_slug_periods(time_threshold, time_gas): # Função para calcular os períodos em que o corpo de slug e a bolha estão presentes
    dt_liquid = [] # Lista para armazenar os períodos de presença de líquido (corpo de slug)
    dt_gas = [] # Lista para armazenar os períodos de presença de gás (bolha)
    
    if len(time_threshold) > 0 and len(time_gas) > 0:
        i_liquid = time_threshold[0] # Inicializa o tempo de início do corpo de slug
        i_gas = time_gas[0] # Inicializa o tempo de início da bolha
        
        if i_liquid > i_gas:# Verifica se o tempo de início do corpo de slug é maior que o tempo de início da bolha
            dt_gas.append(round(i_liquid-0.01-i_gas, 2)) # Calcula e adiciona o período da bolha antes à lista
 
        for j in range(1,len(time_threshold)): # Itera sobre os índices de time_threshold a partir do segundo elemento (o primeiro já foi analisado acima)
            
            if abs(round(time_threshold[j] - time_threshold[j-1], 2)) > 0.01: # Verifica se a diferença entre o tempo atual e o anterior é maior que 0.01 segundos
                f_liquid = time_threshold[j - 1] # Tempo de término do corpo de slug
                dt_liquid.append(round(f_liquid - i_liquid, 2)) # Calcula e adiciona o período do corpo de slug à lista
                f_gas = round(time_threshold[j]-0.01,2) # Término da bolha antes do próximo corpo de slug
                dt_gas.append(round(f_gas - f_liquid-0.01, 2)) # Calcula e adiciona o período da bolha à lista
                i_liquid=time_threshold[j] # Atualiza o tempo de início do próximo corpo de slug
            
        if len(time_gas) > 0: # Verifica se time_gas não está vazio
            f_liquid = time_threshold[-1] # Tempo de término do último corpo de slug
            dt_liquid.append(round(f_liquid - i_liquid, 2)) # Calcula e adiciona o período do último corpo de slug à lista
            i_gas = round(f_liquid + 0.01, 2) # Atualiza o tempo de início da próxima bolha

        if i_gas < time_gas[-1]:  # Verifica se o tempo de início da próxima bolha é menor que o tempo de término da última bolha
                f_gas = time_gas[-1] # Tempo de término da última bolha
                dt_gas.append(round(f_gas - i_gas, 2)) # Calcula e adiciona o período da última bolha à lista

    return dt_liquid, dt_gas # Retorna as listas de períodos de líquido e gás

In [ ]:
def graph_alpha_time(Time_func):
    Time_level = np.array(Time_func) # Converte a lista Time_func em um array numpy
    h, t, alpha_mean = Time_level.T # Extrai as colunas h (altura), t(tempo) e alpha_mean usando transposição .T
    
    ma_escolhido = 10 # Define o tamanho da janela da média móvel ponderada
    alpha = moving_average(alpha_mean, ma_escolhido) # Calcula o valor de alfa ponderado pela média móvel

    threshold = 0.80

    alpha_bin = []  # Lista para armazenar valores binários de alfa
    time_threshold = [] # Lista para armazenar tempos acima do threshold
    time_gas = [] # Lista para armazenar tempos abaixo do threshold

    for i in range(len(alpha)):
        if 1 - alpha[i] >= threshold: # Se 1 - alfa (holdup - nível de líquido) estiver acima do threshold 
            alpha_bin.append(1) # Adiciona 1 à lista de valores binários
            time_threshold.append(t[i]) # Adiciona o tempo ao qual alfa está acima do threshold

        else: # Se alfa estiver abaixo do threshold
            alpha_bin.append(1 - alpha[i]) # Adiciona (1 - alfa) à lista de valores binários
            time_gas.append(t[i]) # Adiciona o tempo ao qual alfa está abaixo do threshold

    if len(time_threshold) > 0:
        dt_liquid, dt_gas = calculate_slug_periods(time_threshold, time_gas)  # Calcula os períodos médios de líquido e gás
        dt_liquid=[x for x in dt_liquid if x != 0]
        dt_gas=[x for x in dt_gas if x != 0]

        if len(dt_liquid) > 0 and len(dt_gas) > 0:

            mean_dt_gas = np.mean(dt_gas) # Calcula a média dos períodos da bolha
            mean_dt_liquid = np.mean(dt_liquid) # Calcula a média dos períodos do corpo de slug

            # Calcula as frequências médias de gás e líquido
            freq_liquid = round(1 / mean_dt_liquid, 3)
            freq_gas = round(1 / mean_dt_gas, 3)

            # Imprime os resultados
            print(f'Períodos de uma bolha: {dt_gas}') 
            print(f'Períodos de um corpo de slug: {dt_liquid}')
            print(f'Período médio da bolha: {mean_dt_gas}')
            print(f'Período médio do corpo de um slug: {mean_dt_liquid}')
            print(f'A frequência média do corpo de slugs é:   {freq_liquid}')
            print(f'A frequência média da bolha é:   {freq_gas}')
            print(f'A frequência média do slug é:   {round(1 / (mean_dt_liquid+mean_dt_gas), 3)}\n')

    # Plotando resultados
    plt.ylim(0, 1) 
    plt.plot(t, 1 - alpha_mean, linewidth=1.5, label='Holdup', color='#5CBEE0', linestyle=':')
    plt.plot(t[1:(len(alpha) + 1)], 1 - alpha, linewidth=1.5, color='black', label=f'Holdup + Média móvel de {ma_escolhido}', linestyle='--')
    plt.plot(t[1:(len(alpha_bin) + 1)], np.array(alpha_bin), linewidth=1.5, color='red', label=f'Holdup + Média móvel de {ma_escolhido} + 'f'Threshold de {threshold}')
    plt.axhline(y=threshold, color='#D1D0D0', linewidth=1.0, linestyle='--')
    plt.ylim(-0.1, 1.1)
    plt.ylabel('Holdup', fontsize=20)
    plt.xlabel(r'Time,\ $t$\ [s]. ', fontsize=20)
    plt.legend(loc='lower left')
    plt.savefig(f"results/Alpha_t.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # PSD
    sampling_rate = 100 # Taxa de amostragem definida como 100 Hz (definida a partir de flow_time)
    frequencies, psd = welch(alpha_mean, sampling_rate, nperseg=400) # Calcula a Densidade Espectral de Potência (PSD) usando o método de Welch
    # Plotando a Densidade Espectral de potência (PSD) vs Frequencias
    plt.figure(figsize=(2, 4))
    plt.xlim(0, 10)
    plt.ylim(0, 0.06)
    plt.plot(frequencies, psd, color='black') # Plota a PSD em função das frequências
    custom_markers_y = [0.00, 0.02, 0.04, 0.06] # Marcadores personalizados para o eixo y
    plt.yticks(custom_markers_y, ['' for _ in custom_markers_y]) # Define os rótulos dos marcadores do eixo y como vazios
    custom_markers_x = [0, 5, 10] # Marcadores personalizados para o eixo x
    plt.xticks(custom_markers_x, ['' for _ in custom_markers_x]) # Define os rótulos dos marcadores do eixo x como vazios
    ax = plt.gca() # Obtém o eixo atual
    ax.spines['top'].set_visible(False) # Torna a linha superior invisível
    ax.spines['right'].set_visible(False) # Torna a linha direita invisível
    ax.spines['left'].set_visible(False) # Torna a linha esquerda invisível
    plt.savefig(f"results/PSD.png", bbox_inches='tight')
    plt.show()
    psd_peak_index = np.argmax(psd) # Encontra o índice do pico da PSD
    psd_peak_frequency = frequencies[psd_peak_index] # Encontra a frequência correspondente ao pico da PSD
    psd_peak_value = psd[psd_peak_index] # Encontra o valor correspondente ao pico da PSD
    print(f'Frequência do pico da PSD: {psd_peak_frequency} Hz') # Imprime a frequência do pico da PSD
    print(f'Valor do pico da PSD: {psd_peak_value}') # Imprime o valor do pico da PSD

    # FFT
    # Criando um vetor de frequencias correspondente ao resultado de FFT
    frequencies = np.fft.fftfreq(len(alpha_mean), 1/sampling_rate) # Calcula as frequências correspondentes aos resultados da FFT
    fft_alpha = np.abs(np.fft.fft(alpha_mean)) # Calcula a magnitude da FFT
    # Plotando a magnitude de FFT vs Frequencias
    plt.figure(figsize=(2, 4))
    plt.xlim(0, 10)
    plt.ylim(0, 60)
    plt.plot(frequencies[frequencies>0], fft_alpha[frequencies>0], color='black') # Plota a magnitude da FFT em função das frequências positivas
    custom_markers_y = [0, 20, 40, 60] # Marcadores personalizados para o eixo y
    plt.yticks(custom_markers_y, ['' for _ in custom_markers_y]) # Define os rótulos dos marcadores do eixo y como vazios
    custom_markers_x = [0, 5, 10] # Marcadores personalizados para o eixo x
    plt.xticks(custom_markers_x, ['' for _ in custom_markers_x]) # Define os rótulos dos marcadores do eixo x como vazios
    ax = plt.gca() # Obtém o eixo atual
    ax.spines['top'].set_visible(False) # Torna a linha superior invisível
    ax.spines['right'].set_visible(False) # Torna a linha direita invisível
    ax.spines['left'].set_visible(False) # Torna a linha esquerda invisível
    plt.grid(False)
    plt.savefig(f"results/FFT.png", bbox_inches='tight')
    plt.show()
    fft_peak_index = np.argmax(fft_alpha[frequencies > 0]) # Encontra o índice do pico da FFT para as frequências positivas
    fft_peak_frequency = frequencies[frequencies > 0][fft_peak_index] # Encontra a frequência correspondente ao pico da FFT
    fft_peak_value = fft_alpha[frequencies > 0][fft_peak_index] # Encontra o valor correspondente ao pico da FFT
    print(f'Frequência do pico da FFT: {fft_peak_frequency} Hz') # Imprime a frequência do pico da FFT
    print(f'Valor do pico da FFT: {fft_peak_value}')

graph_alpha_time(Time_func)

#### Todo o comprimento da tubulação

In [ ]:
space_func = [] # Lista vazia para armazenar os valores de espaço

def Level_space(Z_values): # Esta função recebe um DataFrame 'Z_values' contendo os dados para a posição escolhida
    alpha = [row[7] for row in Z_values[0:]] # Extrai a coluna 7 de Z_values e armazena em 'alpha'
    row = None  # Inicializa row como None

    for i, row in enumerate(Z_values[0:], start=0): # Itera sobre as linhas de Z_values com um índice 'i' começando em 1
        if round(row[7], 1) == 1.00 and row[7] != Z_values[i - 1][7]: # Se o valor arredondado de row[7] for 1.00 e for diferente do valor anterior
            break
        elif i == len(Z_values) - 1: # Se for a última iteração do loop
            break
        
    if row is not None: # Se 'row' não for None (ou seja, o loop encontrou uma linha que satisfaz a condição)
        # Adiciona uma tupla à lista 'space_func' contendo:
        # - float(row[2]): posição radial
        # - float(row[3]): Tempo
        # - float(np.mean(alpha)): média dos valores de 'alpha' na direção radial para uma posição axial fixa
        space_func.append(tuple([float(row[2]), float(row[3]),float(np.mean(alpha))])) # Adiciona uma tupla à lista Time_func contendo os valores de posição radial, posição axial e média dos níveis

In [ ]:
def calculate_slug_lenght(space_threshold,space_gas): # Função para calcular os períodos em que o corpo de slug e a bolha estão presentes
    ds_liquid = [] # Lista para armazenar os comprimentos dos corpos de slug
    ds_gas = [] # Lista para armazenar os comprimentos das bolhas

    if space_threshold and space_gas: # Garante que space_threshold e space_gas tenham pelo menos um elemento antes de tentar acessá-los
        i_liquid = space_threshold[0] # Inicializa a posição de início do corpo de slug
        i_gas = space_gas[0] # Posição de início da bolha
        ds_gas.append(i_liquid-i_gas) #  Calcula e adiciona o comprimento da bolha à lista

    for j in range(len(space_threshold)): # Loop que percorre todos os elementos da lista space_threshold
        if (space_threshold[j] - space_threshold[j - 1]) >= 0.02: # Verifica se a diferença entre o valor atual e o anterior em space_threshold é maior ou igual a 0.02
            f_liquid = space_threshold[j - 1] # Posição de término do corpo de slug
            ds_liquid.append(f_liquid - i_liquid) # Calcula e adiciona o comprimento do corpo de slug à lista
            f_gas = space_threshold[j] # Atualiza a posição de início da bolha
            ds_gas.append(f_gas - i_liquid) # Calcula e adiciona o comprimento da bolha à lista
            i_liquid = f_gas # Atualiza a posição de início do corpo de slug

        if j == len(space_threshold) - 1:  # Se estivermos no último índice em space_threshold
            f_liquid = space_threshold[j-1] # Posição de término do corpo de slug
            ds_liquid.append(f_liquid - i_liquid) # Calcula e adiciona o período do corpo de slug restante à lista
            i_gas=space_threshold[j-1] # Atualiza a posição de início da bolha
            f_gas=space_gas[-1] # Posição de término da bolha (último valor de space_gas)
            ds_gas.append(f_gas-i_gas) # Calcula e adiciona o comprimento da bolha à lista

    return ds_liquid, ds_gas # Retorna as listas de comprimentos dos corpos de slug e das bolhas

In [ ]:
image_paths = [] # Lista para armazenar os caminhos das imagens geradas

def graph_alpha(space_func, current_time):
    Space_level = np.array(space_func) # Converte a lista space_func em um array numpy
    h, z, alpha_mean = Space_level.T # Extrai as colunas h (altura), z(posição axial) e alpha_mean usando transposição .T

    ma_escolhido = 50 # Define o tamanho da janela da média móvel ponderada
    alpha = moving_average(alpha_mean, ma_escolhido) # Calcula o valor de alfa ponderado pela média móvel

    threshold = 0.80 # Define o limiar de holdup para classificação de slugs

    alpha_bin = []  # Lista para armazenar valores binários de alfa
    space_threshold = [] # Lista para armazenar posições onde alfa está acima do threshold
    space_gas = [] # Lista para armazenar posições onde alfa está abaixo do threshold

    for i in range(len(alpha)): 
        if 1 - alpha[i] >= threshold: # Se 1 - alfa (holdup - nível de líquido) estiver acima do threshold 
            alpha_bin.append(1) # Adiciona 1 à lista de valores binários
            space_threshold.append(z[i]) # Adiciona a posição onde alfa está acima do threshold
        else: # Se alfa estiver abaixo do threshold
            alpha_bin.append(1 - alpha[i]) # Adiciona (1 - alfa) à lista de valores binários
            space_gas.append(z[i]) # Adiciona a posição onde alfa está abaixo do threshold

    ds_liquid, ds_gas = calculate_slug_lenght(space_threshold, space_gas)  # Calcula os comprimentos médios de líquido e gás

    mean_ds_gas = np.mean(ds_gas) # Calcula a média dos comprimentos da bolha
    mean_ds_liquid = np.mean(ds_liquid) # Calcula a média dos comprimentos do corpo de slug

    # Imprime os resultados
    #print(f'Comprimentos de uma bolha: {ds_gas}')
    #print(f'Comprimentos de um corpo de slug: {ds_liquid}')
    #print(f'Comprimento médio da bolha: {mean_ds_gas}')
    #print(f'Comprimento médio do corpo de um slug: {mean_ds_liquid}')
    #print(f'Comprimento da maior bolha: {s_gas}')

    # Plotando resultados
    plt.plot(z[1:(len(alpha) + 1)], 1 - alpha, linewidth=1.5, color='black', label=f'Holdup + Média móvel de {ma_escolhido}', linestyle='--')
    plt.plot(z[1:(len(alpha_bin) + 1)], np.array(alpha_bin), linewidth=1.5, color='red', label=f'Holdup + Média móvel de {ma_escolhido} + 'f'Threshold de {threshold}')
    plt.axhline(y=threshold, color='#D1D0D0', linewidth=1.0, linestyle='--')
    plt.ylim(-0.1, 1.1)
    plt.ylabel('Holdup', fontsize=20)
    plt.xlabel(r'Axial Position,\ $z$\ [m]. ', fontsize=20)
    plt.legend(loc='lower left')
    plt.savefig(f"results/Alpha_s_{current_time:.2f}.png", dpi = 300, bbox_inches='tight')
    #plt.show()
    plt.close()  # Fecha o gráfico para liberar memória

    image_paths.append(f"results/Alpha_s_{current_time:.2f}.png") # Adiciona o caminho da imagem à lista
    
    return mean_ds_gas, mean_ds_liquid # Retorna o comprimento da maior bolha

In [ ]:
bubble_lengths = []  # Lista para armazenar os comprimentos das maiores bolhas
slug_body_lengths = []
time_values = []  # Lista para armazenar os valores de tempo

current_time = 5  # Tempo inicial
time_step = 0.01  # Passo de tempo

# Loop sobre todos os arquivos de dados
for file_path in data_time:
    space_func = [] # Lista para armazenar os valores de espaço processados
    data = pd.read_csv(file_path) # Lê o arquivo CSV em um DataFrame do Pandas
    
    data_m = data.to_numpy() # Converte o DataFrame em uma matriz NumPy
    cell_id, X_all, Y_all, Z_all, U, V, W, Phase = np.array(data_m).T # Transpõe a matriz e descompacta as colunas

    Y_list = sorted(np.array(list(set(Y_all))))  # Cria uma lista ordenada dos valores únicos de Y
    Z_list = sorted(np.array(list(set(Z_all))))  # Cria uma lista ordenada dos valores únicos de Z

    data_m = sorted(data_m, key=lambda a_entry: (a_entry[3], a_entry[2])) # Ordena os dados por Z (índice 3) e depois por Y (índice 2)

    # Extrai os níveis de fase dos dados
    for i in Z_list:
        Z_pos = i # Posição atual de Z
        Z_values=[] # Lista para armazenar os valores de Z

        for row in data_m:
            if row[3] == Z_pos: # Verifica se a posição Z da linha atual corresponde à posição Z atual
                Z_values.append(row) # Adiciona a linha atual à lista Z_values

        Level_space(Z_values) # Chama a função Level_space para processar os valores de Z

    # Verifica se existem dados válidos antes de calcular
    if len(space_func) > 0:
        # Calcula o comprimento da maior bolha
        s_gas, s_liquid = graph_alpha(space_func, current_time) # Chama a função graph_alpha e armazena o comprimento da maior bolha
        bubble_lengths.append(s_gas) # Adiciona o comprimento da maior bolha à lista bubble_lengths
        slug_body_lengths.append(s_liquid)
        time_values.append(current_time) # Adiciona o tempo atual à lista time_values
        current_time += time_step # Incrementa o tempo atual pelo passo de tempo
        image_path = f"results/Alpha_s_{current_time}.png" # Define o caminho do arquivo da imagem
        image_paths.append(image_path) # Adiciona o caminho da imagem à lista image_paths

In [ ]:
import math
new_bubble_lengths=[x for x in bubble_lengths if not math.isnan(x) and x != 0]
new_slug_body_lengths=[x for x in slug_body_lengths if not math.isnan(x) and x != 0]

print(f'Média bolha: {np.mean(new_bubble_lengths)}') # Exibe o valor da média do comprimento das maiores bolhas
print(f'Desvio padrão bolha: {np.std(new_bubble_lengths)}') # Exibe o valor do desvio padrão do comprimento das maiores bolhas
print(f'Média corpo de slug: {np.mean(new_slug_body_lengths)}') # Exibe o valor da média do comprimento das maiores bolhas
print(f'Desvio padrão corpo de slug:{np.std(new_slug_body_lengths)}') # Exibe o valor do desvio padrão do comprimento das maiores bolhas

In [ ]:

# Plotando os comprimentos das maiores bolhas ao longo do tempo
plt.plot(time_values, bubble_lengths)
plt.xlabel(r'Time,\ $t$\ [s]. ', fontsize=20)
plt.ylabel(r'Comprimento da bolha,\ $L_{bolha}$\ [m]. ', fontsize=20)
plt.savefig(f"results/comprimento_vs_tempo.png", bbox_inches='tight')
plt.show()

In [ ]:
existent_paths = [path for path in image_paths if os.path.exists(path)] # Verifica se os caminhos das imagens na lista image_paths existem
image_paths = existent_paths # Atualiza a lista original image_paths apenas com os caminhos que realmente existem

In [ ]:
# Cria um clip de vídeo a partir das imagens
video_clip = ImageSequenceClip(image_paths, fps=10)  
# Salva o vídeo
output_video_path = "results/output_video.mp4"  # Nome do arquivo de saída do vídeo
video_clip.write_videofile(output_video_path)
video_clip.write_gif("results/alpha.gif")